In [1]:
import numpy as np
import pandas as pd
import sklearn

import lightgbm as lgb
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import statsmodels.api as sm
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore')

%load_ext autotime

time: 191 µs (started: 2022-05-08 17:06:26 +04:00)


In [2]:
filepath = "../Data/TrainDataMf.csv"

train_data = pd.read_csv(filepath)

filepath = "../Data/TestDataMf.csv"

test_data = pd.read_csv(filepath)

time: 5.14 s (started: 2022-05-08 17:06:26 +04:00)


In [3]:
# Dropping "Lead Flag" and "ZIPCode" to run LightGBM model on the train and test datasets

x_train = train_data.drop(['Lead Flag', 'ZIPCode'], axis=1)
y_train = train_data['Lead Flag']

x_test = test_data.drop(['Lead Flag', 'ZIPCode'], axis=1)
y_test = test_data['Lead Flag']

time: 372 ms (started: 2022-05-08 17:06:31 +04:00)


In [4]:
# In this dictionary the results of LightGBM will be added, then dataframe will be made based on this

compare_results_lgbm = {'learning_rate':[], 'max_depth':[], 'score_train':[], 'score_test': []}

time: 465 µs (started: 2022-05-08 17:06:31 +04:00)


In [5]:
# Running nested for loops to put values in the parameters of LightGBM, run the model, save the train and test scores. 

results = []
for i in np.arange (0.1, 0.9,0.1):
    for j in range(-5,5, 2):
        model = lgb.LGBMClassifier(learning_rate=i, max_depth=j, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric='AUC', verbose=20)
        
        compare_results_lgbm['learning_rate'].append(i)
        compare_results_lgbm['max_depth'].append(j)

        Y0_lgbm=model.predict_proba(x_train)[:,1]
        Y1_lgbm=model.predict_proba(x_test)[:,1]
        
        compare_results_lgbm['score_train'].append(roc_auc_score(y_train, Y0_lgbm))
        compare_results_lgbm['score_test'].append(roc_auc_score(y_test, Y1_lgbm))

[20]	training's auc: 0.735908	training's binary_logloss: 0.0495738
[40]	training's auc: 0.77044	training's binary_logloss: 0.0475879
[60]	training's auc: 0.799294	training's binary_logloss: 0.0459133
[80]	training's auc: 0.820631	training's binary_logloss: 0.0445185
[100]	training's auc: 0.8394	training's binary_logloss: 0.0431815
[20]	training's auc: 0.735908	training's binary_logloss: 0.0495738
[40]	training's auc: 0.77044	training's binary_logloss: 0.0475879
[60]	training's auc: 0.799294	training's binary_logloss: 0.0459133
[80]	training's auc: 0.820631	training's binary_logloss: 0.0445185
[100]	training's auc: 0.8394	training's binary_logloss: 0.0431815
[20]	training's auc: 0.735908	training's binary_logloss: 0.0495738
[40]	training's auc: 0.77044	training's binary_logloss: 0.0475879
[60]	training's auc: 0.799294	training's binary_logloss: 0.0459133
[80]	training's auc: 0.820631	training's binary_logloss: 0.0445185
[100]	training's auc: 0.8394	training's binary_logloss: 0.0431815
[

[80]	training's auc: 0.757629	training's binary_logloss: 0.0497706
[100]	training's auc: 0.763737	training's binary_logloss: 0.0495679
[20]	training's auc: 0.722086	training's binary_logloss: 0.0589822
[40]	training's auc: 0.724212	training's binary_logloss: 0.0636796
[60]	training's auc: 0.724258	training's binary_logloss: 0.063975
[80]	training's auc: 0.722742	training's binary_logloss: 0.0686566
[100]	training's auc: 0.718204	training's binary_logloss: 0.0710543
[20]	training's auc: 0.722086	training's binary_logloss: 0.0589822
[40]	training's auc: 0.724212	training's binary_logloss: 0.0636796
[60]	training's auc: 0.724258	training's binary_logloss: 0.063975
[80]	training's auc: 0.722742	training's binary_logloss: 0.0686566
[100]	training's auc: 0.718204	training's binary_logloss: 0.0710543
[20]	training's auc: 0.722086	training's binary_logloss: 0.0589822
[40]	training's auc: 0.724212	training's binary_logloss: 0.0636796
[60]	training's auc: 0.724258	training's binary_logloss: 0.06

In [6]:
# Making dataframe from dictionary. Getting the best test score for LightGBM and corresponding parameters.

compare_results_lgbm = pd.DataFrame.from_dict(compare_results_lgbm)
compare_results_lgbm[compare_results_lgbm['score_test'] == max(compare_results_lgbm['score_test'])] #0.1 -1

,learning_rate,max_depth,score_train,score_test
4,0.1,3,0.740066,0.716525


time: 28.6 ms (started: 2022-05-08 17:12:35 +04:00)


In [7]:
# In this dictionary the results of XGBoost will be added, then dataframe will be made based on this

compare_results_xgb = {'learning_rate':[], 'max_depth':[], 'score_train':[], 'score_test': []}

time: 225 µs (started: 2022-05-08 17:12:35 +04:00)


In [8]:
# Running nested for loops to put values in the parameters of XGBoost, run the model, save the train and test scores. 

results = []
for i in np.arange (0.1, 0.9,0.2):
    for j in range(1,10, 4):
        print('learning rate: ', i, " max_depth: ", j)
        model = XGBClassifier(learning_rate=i, max_depth=j, random_state=42)
        model.fit(x_train, y_train, eval_set=[(x_train, y_train)], eval_metric='auc', verbose=20) 
        
        compare_results_xgb['learning_rate'].append(i)
        compare_results_xgb['max_depth'].append(j)
        
        
        Y0_xgb=model.predict_proba(x_train)[:,1]
        Y1_xgb=model.predict_proba(x_test)[:,1]
        
        compare_results_xgb['score_train'].append(roc_auc_score(y_train, Y0_xgb))
        compare_results_xgb['score_test'].append(roc_auc_score(y_test, Y1_xgb))

learning rate:  0.1  max_depth:  1
[0]	validation_0-auc:0.59582
[20]	validation_0-auc:0.59582
[40]	validation_0-auc:0.64965
[60]	validation_0-auc:0.67234
[80]	validation_0-auc:0.68408
[99]	validation_0-auc:0.69082
learning rate:  0.1  max_depth:  5
[0]	validation_0-auc:0.63141
[20]	validation_0-auc:0.69508
[40]	validation_0-auc:0.71745
[60]	validation_0-auc:0.74885
[80]	validation_0-auc:0.77041
[99]	validation_0-auc:0.78294
learning rate:  0.1  max_depth:  9
[0]	validation_0-auc:0.63406
[20]	validation_0-auc:0.73561
[40]	validation_0-auc:0.83990
[60]	validation_0-auc:0.90673
[80]	validation_0-auc:0.92877
[99]	validation_0-auc:0.93888
learning rate:  0.30000000000000004  max_depth:  1
[0]	validation_0-auc:0.59582
[20]	validation_0-auc:0.67159
[40]	validation_0-auc:0.69687
[60]	validation_0-auc:0.70647
[80]	validation_0-auc:0.71069
[99]	validation_0-auc:0.71380
learning rate:  0.30000000000000004  max_depth:  5
[0]	validation_0-auc:0.63141
[20]	validation_0-auc:0.74348
[40]	validation_0-

In [9]:
# Making dataframe from dictionary. Getting the best test score for XGBoost and corresponding parameters.

compare_results_xgb = pd.DataFrame.from_dict(compare_results_xgb)
compare_results_xgb[compare_results_xgb['score_test'] == max(compare_results_xgb['score_test'])] #0.2 -1

,learning_rate,max_depth,score_train,score_test
9,0.7,1,0.721153,0.715388


time: 8.57 ms (started: 2022-05-08 17:28:53 +04:00)


In [10]:
# Running Logistic Regression without regularization 

logreg = LogisticRegression(random_state=42)

logreg.fit(x_train,y_train)

Y0_logreg=logreg.predict_proba(x_train)[:,1]
Y1_logreg=logreg.predict_proba(x_test)[:,1]

print("ROC_AUC Train:", roc_auc_score (y_train, Y0_logreg))
print ("ROC_AUC Test:", roc_auc_score(y_test, Y1_logreg))

ROC_AUC Train: 0.5477270801925344
ROC_AUC Test: 0.5494109220982586
time: 9.88 s (started: 2022-05-08 17:28:53 +04:00)


In [11]:
# Running Logistic Regression with regularization. Better results than without regularization

logreg=LogisticRegression(C=1,penalty='l1',solver="liblinear", random_state=42)

logreg.fit(x_train,y_train)

Y0_logreg=logreg.predict_proba(x_train)[:,1]
Y1_logreg=logreg.predict_proba(x_test)[:,1]

print("ROC_AUC Train:", roc_auc_score (y_train, Y0_logreg))
print ("ROC_AUC Test:", roc_auc_score(y_test, Y1_logreg))

ROC_AUC Train: 0.7041530313159494
ROC_AUC Test: 0.7078894038471863
time: 30.8 s (started: 2022-05-08 17:29:03 +04:00)
